In [ ]:
import numpy as np
import os,sys
import six.moves.urlib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import imageio
imageio.plugins.ffmpeg.download()
#import datetime
from datetime import datetime
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

#This is needed since the notebook is store in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf._version_ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
    

In [ ]:
RESEARCH_PATH = '../../tf-models/research'
MODELS_PATH = '../../tf-models/research/object_detection'
sys.path.append(RESEARCH_PATH)
sys.path.append(MODELS_PATH)


Envarionment Setup

In [ ]:
#this is needed to display the images.
%matplotlib inline

Object Detection imports

In [ ]:
from utils import label_map_util
from utils import visualization_utils as vis_util

Model preparation
Variabel
any model exported using the export_inference_graph.py tool can be loaded here simply by changing PATH_TO_CKPT to point to a new.pb file.
By default we use an "SSD with Mobilenet" model here. see the detection model zoo for a list of other models that can be run out-of-the-box with varying speed and accuracies.

In [ ]:
#What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#path to frozen detection graph. THis is the actual model that is used for the object detection
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

#list of the strings that is used to add correct label for each box
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

Download models

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())
        

Load a (frozen) Tensorflow model into memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

Loading Label map
label maps map indicates to category names, so that when our convolutin network predict 5. we know that this corresponds to airplane. here we use internal utility functions, but anything that returns a directionary maping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.creat_category_index(categories)

Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

Detection

In [ ]:
# For the sake of simplicity we will use only 2 image
#image1.jpg
#image2.jpg
#if you want to test the code with your images, just add path to the image to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'beach{}.jpg'.format(i)) for i in range(1,3) ]

#Size, in inches, of the output images.
IMAGE_SIZE = (12,8)


Visualization of teh result of a detection 

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_score:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        input_video = 'traffic' # traffic itu nama vidionya yg mau di detect objek didalamnya
        video_reader = imageio.get_reader('%s.mp4'%input_video) 
        video_writer = imageio.get_writer('%s_annotated.mp4'% input_video,fps=10) # hasilnya nnti akan tersafe dengan nama trafic_annotated.mp4
        
        #loop through and process each frame
        t0 = datetime.now()
        n_frames = 0
        for frame in video_reader:
            image_np = frame
            n_frames+=1
            
            #expand dim since the expects images to have shape[1,None]
            image_np_expanded = np.expand_dims(image_np,axis=0)
            #actual detection
            (boxes,scores,classes,num) = sess.run([detection_boxes,detection_scores,detection_classes,num_detections],feed_) # tdk lengkappi code trakhirnya
            # Visualization
            vis_util.visualize_boxes_and_labels_on_image_array(image_np,np.squeeze(boxes),np.squeeze(classes).astype(np.in)) # tdk lengkap juga codenya 
            #video writer
            video_writer.append_data(image_np)
            
        fps=n_frames/(datetime.now()-t0).total_seconds()
        print("Frames Processed: %s,Speed:%s fps"%(n_frames,fps))
        
        #cleanup
        video_writer.close()
        